In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
x_train_path = "data/X_train_Hi5.csv"
X_train = pd.read_csv(x_train_path)

In [ ]:
X_train.head(15)

In [ ]:
X_train.info()


In [ ]:
X_train[["piezo_groundwater_level_category"]]

In [ ]:
X_train.sort_values(["piezo_groundwater_level_category"])


In [ ]:
X_train=X_train.rename({"piezo_groundwater_level_category":"target"},axis=1)
X_train.head()

In [ ]:
X_train.sort_values(["target"])

In [ ]:
X_train[['piezo_station_department_code','target']].sort_values(["target"])



In [ ]:
humid_40=X_train.loc[X_train["target"]=="Low","meteo_humidity_duration_below_40%"]
humid_40.dropna()
humid_40.plot()

In [ ]:
humid_80=X_train.loc[X_train["target"]=="High","meteo_humidity_duration_above_80%"]
humid_80.plot()

In [ ]:
howwet=X_train.loc[X_train["target"]=="Very Low","meteo_wetting_duration"]
howwet.dropna()


In [ ]:
level_rain=X_train.loc[X_train["target"]=="Very Low","meteo_rain_height"]
level_rain.plot()

In [ ]:

target_choice = ["Very Low", "Low", "Average", "High", "Very High"]


fig, axes = plt.subplots(5, 1, figsize=(20, 20))  

for i in range(5):  
    usage_type = X_train.loc[X_train["target"] == target_choice[i], "prelev_usage_label_2"]
    usage_type = usage_type.dropna()  
    

    percentages = usage_type.value_counts(normalize=True)
    
    percentages.plot.pie(ax=axes[i], autopct='%1.1f%%',startangle=90, title=f"Usage of the water for stations\nwith groundwater level {target_choice[i]}")
 
plt.tight_layout()
plt.show()



In [ ]:
df_elab= X_train[["hydro_hydro_quantity_elab","hydro_observation_result_elab","hydro_observation_date_elab","hydro_station_code"]]
df_elab.sort_values(["hydro_station_code"])

In [ ]:
#Convertir les dates en nombres de jours
def date_counter(date):
    y_m_d=date.split("-")
    y_m_d=[int(dates) for dates in y_m_d]
    jours=(y_m_d[0]-2020)*365+(y_m_d[1]-1)*30+y_m_d[2]
    return jours

date_counter("2023-01-01")





In [ ]:
df_elab["days"]=df_elab["hydro_observation_date_elab"].apply(date_counter)
df_elab.sort_values(["hydro_station_code","days"])


In [ ]:
df_elab_stations=df_elab.loc[df_elab["hydro_station_code"]=="A022020001" ,["hydro_hydro_quantity_elab","hydro_observation_result_elab","days"]] #only the "elab" variables
df_elab_stations

df_elab_station1=df_elab_stations.loc[df_elab["hydro_hydro_quantity_elab"]=="QmJ"] #only the daily measurements


df_elab_station1.plot(y="hydro_observation_result_elab",x="days",title="Évolution de hydro_quantity_elab sur une station aléatoire")